In [ ]:
import pickle
import pandas as pd
import numpy as np

In [ ]:
train = pickle.load(open('/tmp/citi/train_sample.pckl/train_sample.pckl', 'rb'))

In [ ]:
train['delivery']=train['delivery_available'].map(lambda x: 1 if x else 0)

In [ ]:
train['delivery_available'].value_counts()

In [ ]:
train['delivery'].value_counts()

In [ ]:
train['is_new']=train['description'].map(lambda x: 'новый'  in x.lower() or \
                                                    'новая' in  x.lower() or \
                                                    'новое' in  x.lower() or 'new' in x.lower())

In [ ]:
train['lat'] = train['location'].map(lambda x: x['latitude'])
train['lon'] = train['location'].map(lambda x: x['longitude'])
train['is_verygood'] = train['description'].map(lambda x: 'отличн'  in x.lower() or \
                                                          'perfect' in  x.lower() or \
                                                          'идеал'   in  x.lower())

train['is_free'] = train['description'].map(lambda x: 'бесплатно' in x.lower() or \
                                                      'даром' in x.lower() or \
                                                      'отдам за киндер' in x.lower())

In [ ]:
train['is_used'] = train['description'].map(lambda x: 'б/у' in x.lower() or  ' бу ' in x.lower())

In [ ]:
train['is_good']= train['description'].map(lambda x : 'хорош' in x.lower())

In [ ]:
train['is_iphone'] = (train['description'].map(lambda x: 'iphone' in x.lower() or 'айфон' in x.lower())) | \
                            (train['name'].map(lambda x: 'iphone' in x.lower() or 'айфон' in x.lower()))

In [ ]:
train['is_set'] = train['description'].map(lambda x: 'комплект' in x.lower())

In [ ]:
train['is_dress'] = train['name'].map(lambda x: 'платье' in x.lower())

In [ ]:
def avg_price_by_cat (df):
    df1= df[['category','subcategory','price']]
    avg_price_by_cat = pd.pivot_table(df1,  values = 'price', index = ['subcategory'], aggfunc=np.mean)
    avg_price_by_cat.columns = ['avg_price_by_cat']
    return avg_price_by_cat


In [ ]:
avg_price_by_cat= avg_price_by_cat(train)

In [ ]:
train2 = train.merge(avg_price_by_cat, how='inner', on='subcategory')

In [ ]:
data = train2[['category', 'subcategory', 'price', 'lat', 'lon', 'delivery', 'is_iphone', 'is_good', 'is_verygood', \
              'is_new', 'is_free', 'is_used', 'is_set','avg_price_by_cat', 'payment_available','is_dress'           ]]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('price', axis=1),
    data['price'],
    test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
mean_squared_log_error(y_test, rf.predict(X_test))**0.5
# 2.461184253128399
# 2.453224288168561
# 2.161305024494273